<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
 

<strong>Objectives:</strong>
+ Entender la variable Ventas
+ Descartar Productos descatalogados
    
    
<strong>Conclusions:</strong>

   
</div>

In [1]:
import os, sys

def get_root_path(search_word):
    """ Búsqueda hacia atrás de un directorio con nombre search_word. """
    cwd = os.getcwd()
    while str(os.getcwd()).split('/')[-1] != search_word:
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path('TFM_StockForecast')

sys.path.insert(0, ROOT_DIR) # Poner en el primer lugar del PATH (evitar paquetes con mismos nombres en otros directorios)

In [2]:
import pandas as pd
import numpy as np
from src.load_data import load_csv

In [3]:
df = load_csv("ventas")

df

===================  LOAD DATASET ventas ===================
Reading CSV in data/raw/01_TablaVentas.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.
============  CLEAN VENTAS  ============
Drop rows with data from products with no stock data or out-of-date (descatalogados). Rows dropped: 14086
Number of dates missings:                                223
Number of dates missings (droping sundays and festivos): 7
UnidadesVentas to integer.
Dataset ventas loaded. Shape: (91571, 3)



,fecha,producto,udsventa
0,2015-06-29,24,1151
1,2015-06-29,18,771
2,2015-06-29,97,201
3,2015-06-29,57,1119
4,2015-06-29,22,423
...,...,...,...
91566,2020-03-21,80,693
91567,2020-03-21,14,6239
91568,2020-03-21,24,2555
91569,2020-03-21,30,246


In [ ]:
df.loc[~df.producto.isin(products.producto)]

## Productos descatalogados

In [ ]:
ventas_byprod = pd.read_csv("data/clean/ventas_byprod.csv")
stock_byprod = pd.read_csv("data/clean/stock_byprod.csv")

ventas_byprod = ventas_byprod.loc[:,["producto","fecha_primera_venta","fecha_ultima_venta"]]
stock_byprod = stock_byprod.loc[:,["producto","fecha_primer_stock","fecha_ultimo_stock"]]
prod = ventas_byprod.merge(stock_byprod, how ='outer', on='producto')

#prod.loc[prod.fecha_ultima_venta < '2020-03-01']

prod.loc[prod.fecha_primer_stock.isna()]

In [ ]:
 = pd.read_csv("ventas_byprod.csv")


ventas.head()

# Cuantos datos por días para cada producto hay en stock

In [ ]:
df_gr = df.groupby(["fecha","producto"])["udsstock"].count().reset_index()

df_gr

# Cuantos datos por dia hay de media para cada producto?

In [ ]:
df_gr_gr = df_gr.groupby("producto")['udsstock'].mean().reset_index()

In [ ]:
df_gr_gr.loc[df_gr_gr.udsstock < 1.97]

# Estudiar time series para un prodcuto en concreto: 38

In [ ]:
df_38 = df.loc[df.producto == '38', ["fecha", "udsstock"]].set_index("fecha")

df_38.head()

In [ ]:
df_38.plot(figsize=(20,10))

In [ ]:
df_38.loc[(df_38.index > '2019-06-20') & (df_38.index < '2019-07-10')].plot(figsize=(20,10), kind='bar')

In [ ]:
ventas_38 = ventas.loc[ventas.producto == '38', ["fecha", "udsventa"]].set_index("fecha")
ventas_38.loc[(ventas_38.index > '2019-06-20') & (ventas_38.index < '2019-07-10')].plot(figsize=(20,10), kind='bar')

In [ ]:
ventas_38.loc[(ventas_38.index > '2019-06-04') & (ventas_38.index < '2019-06-20')]

In [ ]:
df_38.loc[df_38.index < '2019-06-20']

In [ ]:
df_38.median()

# Intervalos sin valor

In [ ]:
# PRINT EDA FOR STOCK DATA
df = pd.read_csv("data/clean/stock_all.csv", parse_dates= ["fecha"])

variable = "udsstock"

df.dtypes

In [ ]:
def get_event_interval(in_df):
    """
    Calculates a new dataframe with the initial time, the final time and the value in the event.
    
    Args:
        df (pd.DataFrame): Streaming dataset where the first column contains the date (in
                           datetime format) and the second column contains the values of each
                           record. Event changes will be specified by changes in the second column.
    Returns:
        pd.DataFrame: a DataFrame with four columns: the initial time, the final time, the 
                      value that the variable catch during the event and the duration of the
                      event in seconds.
    """
    # Ordenamos por la primera columna y copiamos el dataframe
    df = in_df.sort_values(in_df.columns[0]).copy()
    
    inicio_evento = (df.iloc[:,1] != df.iloc[:,1].shift(1))
    t_inicial = df[inicio_evento].iloc[:,0]
    fin_evento = inicio_evento.shift(-1, fill_value=True)
    t_final = df[fin_evento].iloc[:,0]
    value = df[fin_evento].iloc[:,1]
    
    # Calculamos duracion del evento
    duracion = pd.Series(t_final.values - t_inicial.values).dt.total_seconds().astype(np.int)/(24*3600)
    duracion = duracion + 1
    
    # Conformar dataset de salida
    out_df = pd.DataFrame({'start_date': t_inicial.values, 'end_date': t_final.values, 
                           'value': value.values, 'duration': duracion.values})
    
    return out_df

In [ ]:
total_missings = []
num_intervalos = []
media_intervalos = []
num_intervalos_1dia = []
products = df.producto.unique()

for product in products:
    subdf = df.loc[df.producto == product, ["fecha", "udsstock"]]
    df_intervals = get_event_interval(subdf.fillna(0))
    res = df_intervals.loc[df_intervals.value == 0]
    total_missings.append(res.duration.sum())
    media_intervalos.append(res.duration.mean())
    num_intervalos.append(res.duration.count())
    num_intervalos_1dia.append(res.loc[res.duration == 1].duration.count())
    
results = pd.DataFrame({"productos":products,
                        "total_missings": total_missings,
                        "num_intervalos": num_intervalos,
                        "media_intervalos": media_intervalos,
                       "num_intervalos_1dia":num_intervalos_1dia})

results.describe()

### Numero de intervalos totales y numero de intervalos de 1 dia

In [ ]:
results.total_missings.sum()

In [ ]:
results.num_intervalos.sum()

In [ ]:
results.num_intervalos_1dia.sum()

### Numero de missings por intervalo mayor de 5

In [ ]:
results.loc[results.media_intervalos > 5]

In [ ]:
results.loc[results.total_missings == num_intervalos_1dia]